In [1]:
import os
import sys
import warnings
import time

sys.path.insert(0, os.path.abspath(os.path.join("..", "src")))
warnings.simplefilter("ignore")

import tensorflow as tf
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout, Activation, Reshape
from keras.models import Sequential
from keras.callbacks import EarlyStopping, TensorBoard
from keras.optimizers import Nadam
from keras.optimizers.schedules import ExponentialDecay

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print(f"Using device: {physical_devices[0]}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU device found, using CPU")

from models import create_dataset_from_mapping, load_entire

Using device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


# Dataset creation

To speed up further training we'll load entire dataset into memory. `create_dataset_from_mapping` on load reshapes features back to 4, 4, 1536 and fits 2 average poolings on top of it to reduce size to 4 * 1536. Final dataset is 2D array of shape (n_samples, 4 * 1536) - so we have up to now 1 - (1536 * 4) / (150 * 150 * 3) = ~90% reduction of data.

In [2]:
train_dataset, train_size = create_dataset_from_mapping(mapping_file="train_mapping.csv")
validation_dataset, validation_size = create_dataset_from_mapping(mapping_file="validation_mapping.csv")

2024-06-03 11:01:33.002318: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-06-03 11:01:33.002361: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-06-03 11:01:33.002367: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-06-03 11:01:33.002597: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-03 11:01:33.002612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
X_train, y_train = load_entire(train_dataset, validation_size)
X_valid, y_valid = load_entire(validation_dataset, validation_size)

2024-06-03 11:03:03.238319: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
X_valid.shape

TensorShape([21909, 6144])

# Encoding features with autoencoder

In [ ]:
# Define GELU activation function
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

# Define the encoder
encoder = Sequential([
    Dense(6144, input_shape=(6144, )),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(2048),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1024),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(512),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(512),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(512),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3)
])

decoder = Sequential([
    Dense(256, input_shape=(256,)),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(512),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(512),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(512),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1024),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(2048),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
    Dense(6144),
    Activation(gelu),
    BatchNormalization(),
    Dropout(0.3),
])

autoencoder = Sequential([encoder, decoder])

In [ ]:
lr_scheduler = ExponentialDecay(
    initial_learning_rate=0.003,
    decay_steps=10000,
    decay_rate=0.9,
)
optimizer = Nadam(learning_rate=lr_scheduler)
autoencoder.compile(optimizer=optimizer, loss='binary_crossentropy')


early_stopping = EarlyStopping(patience=5, verbose=1)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)
callbacks_list = [early_stopping, tensorboard]

In [ ]:
epochs = 500

autoencoder.fit(X_train, X_train, epochs=epochs, callbacks=callbacks_list, batch_size=16, validation_data=(X_valid, X_valid))

Epoch 1/500


2024-06-03 10:25:27.974471: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [ ]:
root_dir = os.path.abspath(os.path.join(".", ".."))

encoder.save(os.path.join(root_dir, "models", "encoder.h5"))
decoder.save(os.path.join(root_dir, "models", "decoder.h5"))

In [ ]:
X_train_encoded = encoder.predict(X_train)
X_valid_encoded = encoder.predict(X_valid)

In [ ]:
np.save(os.path.join(root_dir, "data", "X_train_encoded.npy"), X_train_encoded)
np.save(os.path.join(root_dir, "data", "X_valid_encoded.npy"), X_valid_encoded)
np.save(os.path.join(root_dir, "data", "y_train.npy"), y_train)
np.save(os.path.join(root_dir, "data", "y_valid.npy"), y_valid)

Final dataset is 2D array of shape (n_samples, 256) - so we have up to now 1 - (256) / (150 * 150 * 3) = ~99.6% reduction of data.